# Connect to ClickHouse Database

In [99]:
!mkdir -p /home/jupyter/datasphere/project/.clickhouse && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document /home/jupyter/datasphere/project/.clickhouse/root.crt && \
chmod 0600 /home/jupyter/datasphere/project/.clickhouse/root.crt

--2024-09-12 11:11:57--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’

/home/jupyter/datas 100%[===================>]   3.50K  --.-KB/s    in 0s      

2024-09-12 11:11:57 (1.09 GB/s) - ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’ saved [3579/3579]



In [100]:
# указываем параметры подключения
CA = '/home/jupyter/datasphere/project/.clickhouse/root.crt' # если вы не меняли путь при скачивании сертификата

VERIFY = True
SECURE = True
USER = 'admin'
PASSWORD = 'hakaton1'
HOST = "rc1d-uq593a42e02ikh2k.mdb.yandexcloud.net" # адрес Clickhouse
PORT = 8443

In [101]:
!pip install clickhouse_connect > /dev/null

In [102]:
# импортируем библиотеку
import clickhouse_connect

# выполняем запрос - в случае успешного выполнения будет показана текущая версия Clickhouse
with clickhouse_connect.get_client(
            host=HOST, port=PORT, username=USER,
            password=PASSWORD, secure=SECURE, verify=VERIFY, ca_cert=CA) as ch_client:
        print(ch_client.command('SELECT version()'))

24.3.10.33


# Web Scraping

In [103]:
!pip install selenium chromedriver_autoinstaller beautifulsoup4 tqdm > /dev/null

In [104]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

In [105]:
import os
dir = "/content/custom_codes"
os.makedirs(dir, exist_ok=True)
prefs = {"download.default_directory" : dir}
chrome_options.add_experimental_option("prefs", prefs);

In [122]:
driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(10)

In [108]:
from bs4 import BeautifulSoup

url = 'https://nlmk-group.github.io/ds-2.0/?path=/docs/components-sidebar-info--docs'

driver.get(url)

soup = BeautifulSoup(driver.page_source, 'lxml')
# print(soup.prettify())

components = list(map(lambda c: c.get_text(), soup.findAll('button', {"data-parent-id": "components"})))
components[:5]

['Accordion', 'Alert', 'AttachFiles', 'Avatar', 'Badge']

In [109]:
docs = {}
completed_set = set()

In [110]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm
import time
import random

sleep_time = [2, 3, 5, 7, 11]

for c_index, c in tqdm(enumerate(components), total=len(components)):
    if c_index in completed_set: continue

    spans_text = []

    while len(spans_text) == 0:
        url_template = 'https://nlmk-group.github.io/ds-2.0/?path=/docs/components-{}-info--docs'
        url = url_template.format(c.lower())

        time.sleep(random.choice(sleep_time))

        driver.get(url)

        iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//iframe[@id='storybook-preview-iframe']")))
        driver.switch_to.frame(iframe)

        # Parse the HTML
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # print(soup.prettify())


        try:
            WebDriverWait(driver, 50).until(
                EC.presence_of_element_located((By.XPATH, "//span[@data-testid='typography-wrapper']"))
            )
        except TimeoutException as e:
            print("Timed out waiting for spans to load")

        spans = soup.findAll('span', {'data-testid': 'typography-wrapper', 'class': '_typography--variant-Body1_1lcvd_131 _typography_1lcvd_1'})

        spans_text = list(map(lambda c: c.get_text().strip(), spans))

    # print(f"{len(spans_text)=}")
    # print('\n'.join(spans_text))
    # print()

    description = spans_text[0]
    spans_text = spans_text[1:]

    code = soup.findAll('div', {'class': 'sp-code-editor'})
    codes_text = [div.find('pre').get_text() for div in code]

    completed_set.add(c_index)

    # print(f"{len(codes_text)=}")
    # print(codes_text[0])
    # print('-' * 100)

    for d_index, data in enumerate(zip(spans_text, codes_text)):
        key = f'{c}:{c_index}:{d_index}'
        if key not in docs:
            value = f"{c}\n\nОписание:\n{description}\n{data[0]}\n\nКод:\n{data[1]}\n"
            docs[key] = value

100%|██████████| 44/44 [07:47<00:00, 10.61s/it]


In [111]:
len(docs)

164

In [112]:
print(list(docs.items())[160][1])

Tooltip

Описание:
Компонент-подсказка, появляющийся при определенном взаимодействии с дочерним элементом компонента (по умолчанию: при наведении курсора). Tooltip рассчитан не только на работу с простой информацией (текст) с единой стилизацией, но и на визуализацию подсказок, содержащих нестандартную информацию (списки, картинки, таблицы).
По умолчанию Тултип не открывается вокруг disabled элементов. Для того чтобы Тултип открывался корректно в этом случае, нужно оборачиваемый элемент обернуть простым элементом-оберткой, например span.

Код:
import { Tooltip, Button } from '@nlmk/ds-2.0';

export default  App = () => (
  <div style={{
  display: 'flex',
  flexDirection: 'column',
  justifyContent: 'center',
  gap: '12px',
  padding: '30px',
  margin: '0 auto',
  height: '180px',
  color: 'var(--text-grey-900)'
}}>
    <Tooltip description="Сюда вы можете добавить текст/подсказу для компонента">
      <span>
        <Button variant="secondary" disabled>
          Наведи на меня курсор!

In [113]:
driver.quit()

# IAM токен для эмбеддингов

In [79]:
# Устанавливаем пакеты, необходимые для обращения к YandexGPT API
%pip uninstall jwt
%pip install PyJWT -U

In [80]:
import requests
import json
import time
import jwt
import os

In [81]:
# Замените <идентификатор_сервисного_аккаунта> на ваше значение
service_account_id = "aje286em9u4ibs8572di"

In [82]:
with open('/content/authorized_key.json') as f:
    authorized_key = json.load(f)

In [83]:
# Замените <идентификатор_ключа> на ваше значение
key_id = "ajeo9id7551hpq15fv3o"
private_key = authorized_key['private_key']

In [84]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
iam_token = x['iamToken']

# Инициализация базы векторных представлений

In [85]:
!pip install -q langchain_community langchain yandexcloud > /dev/null

In [86]:
# llm
folder = 'b1g01stbeeo96j8kfaiv'
model_uri = f'gpt://{folder}/yandexgpt-lite/latest'
model_uri

'gpt://b1g01stbeeo96j8kfaiv/yandexgpt-lite/latest'

In [138]:
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Cоздаем объект YandexGPTEmbeddings для построения векторов с помощью YandexGPT
embeddings = YandexGPTEmbeddings(iam_token=iam_token, folder_id=folder, sleep_interval=0.1)

In [139]:
from langchain_community.vectorstores import Clickhouse, ClickhouseSettings

ch_config = ClickhouseSettings(host=HOST, port=PORT, username=USER, password=PASSWORD)
docsearch = Clickhouse(embedding=embeddings, config=ch_config, verify=VERIFY, ca_cert=CA)
docsearch

default.langchain @ rc1d-uq593a42e02ikh2k.mdb.yandexcloud.net:8443

username: admin

Table Schema:
---------------------------------------------------
|id                      |Nullable(String)        |
|document                |Nullable(String)        |
|embedding               |Array(Float32)          |
|metadata                |Object('json')          |
|uuid                    |UUID                    |
---------------------------------------------------

# Очистить базу

In [114]:
docsearch.drop()

# Заполнить базу

In [116]:
from langchain_core.documents import Document

documents = [Document(page_content=v) for k, v in docs.items()]
ids = [k for k, v in docs.items()]

ids[:2], documents[:2]

(['Accordion:0:0', 'Accordion:0:1'],
 [Document(page_content='Accordion\n\nОписание:\nКомпонент "аккордеон" предоставляет большие объемы контента в ограниченном пространстве с помощью пошагового раскрытия. Заголовок обеспечивает пользователю общий обзор содержимого, позволяя решить, какие разделы читать.\n\n        Аккордеоны могут сделать обработку информации и поиск более эффективными. Однако они скрывают контент от пользователей, и важно учесть, что пользователь может не заметить или не прочитать весь включенный контент. Если пользователь, вероятно, должен прочесть весь контент, не используйте аккордеон, так как это добавляет дополнительный клик; вместо этого используйте полноценную прокручиваемую страницу с обычными заголовками.\nАккордеон по умолчанию\n\nКод:\nimport { Accordion } from \'@nlmk/ds-2.0\';\n\nconst items = [\n    {\n      id: 1,\n      title: \'First accordion item\',\n      content: \'Lorem ipsum\',\n      initiallyExpanded: true\n    },\n    {\n      id: 2,\n      

In [117]:
docsearch.add_documents(documents=documents, ids=ids)

Inserting data...: 100%|██████████| 164/164 [00:02<00:00, 68.28it/s]


['Accordion:0:0',
 'Accordion:0:1',
 'Accordion:0:2',
 'Accordion:0:3',
 'Alert:1:0',
 'Alert:1:1',
 'Alert:1:2',
 'Alert:1:3',
 'AttachFiles:2:0',
 'AttachFiles:2:1',
 'AttachFiles:2:2',
 'AttachFiles:2:3',
 'AttachFiles:2:4',
 'AttachFiles:2:5',
 'AttachFiles:2:6',
 'AttachFiles:2:7',
 'Avatar:3:0',
 'Avatar:3:1',
 'Avatar:3:2',
 'Avatar:3:3',
 'Avatar:3:4',
 'Avatar:3:5',
 'Badge:4:0',
 'Badge:4:1',
 'Badge:4:2',
 'Badge:4:3',
 'Box:5:0',
 'Breadcrumbs:6:0',
 'Breadcrumbs:6:1',
 'Breadcrumbs:6:2',
 'Button:7:0',
 'Button:7:1',
 'Button:7:2',
 'Button:7:3',
 'Button:7:4',
 'Card:8:0',
 'Card:8:1',
 'Checkbox:9:0',
 'Checkbox:9:1',
 'Checkbox:9:2',
 'Checkbox:9:3',
 'Chip:10:0',
 'Chip:10:1',
 'Chip:10:2',
 'Chip:10:3',
 'Chip:10:4',
 'Chip:10:5',
 'DatePicker:11:0',
 'DatePicker:11:1',
 'DatePicker:11:2',
 'DatePicker:11:3',
 'Divider:12:0',
 'Divider:12:1',
 'Divider:12:2',
 'Divider:12:3',
 'Divider:12:4',
 'Divider:12:5',
 'DragAndDrop:13:0',
 'Drawer:14:0',
 'Drawer:14:1',
 'Drop

# Add custom components example

In [123]:
from bs4 import BeautifulSoup

url = 'https://github.com/Hakaton-NLMK/frontend/tree/main/src/examples'

driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
# print(soup.prettify())

table = soup.find('table', {'class': 'Box-sc-g0xbh4-0'})

tbody = soup.find('tbody')

custom_code_components = []

tr = tbody.find_all('tr', {'class': 'react-directory-row'})
for t in tr:
  td = t.find('td', {'class': 'react-directory-row-name-cell-large-screen'})
  custom_code_components.append(td.get_text())

custom_code_components

['FullPage.tsx', 'LoginForm.tsx', 'NavigationPanel.tsx', 'ProductCard.tsx']

In [124]:
descriptions = [
  "Создай страницу с Defoult Sidebar слева, Header Notification сверху страницы, далее ниже Tabs Default, ниже Defoult Input, ниже Defoult Button.",
  "Создай форму входа с полями для email и пароля и кнопкой 'Войти'.",
  "Создай панель навигации с логотипом и тремя ссылками: 'Главная', 'О нас', 'Контакты'.",
  "Создай карточку продукта с изображением, названием и ценой.",
]

name2description = { k: v for k, v in zip(custom_code_components, descriptions) }
name2description

{'FullPage.tsx': 'Создай страницу с Defoult Sidebar слева, Header Notification сверху страницы, далее ниже Tabs Default, ниже Defoult Input, ниже Defoult Button.',
 'LoginForm.tsx': "Создай форму входа с полями для email и пароля и кнопкой 'Войти'.",
 'NavigationPanel.tsx': "Создай панель навигации с логотипом и тремя ссылками: 'Главная', 'О нас', 'Контакты'.",
 'ProductCard.tsx': 'Создай карточку продукта с изображением, названием и ценой.'}

In [125]:
from selenium.webdriver.common.by import By
import time

files = os.listdir(dir)

for i, c in enumerate(custom_code_components):
    # if i == 1: break
    if c in files: continue

    c_url = f'{url}/{c}'
    driver.get(c_url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # print(soup.prettify())

    # download_source_button = soup.find('button', {'data-testid': 'download-raw-button'})

    download_source_button = driver.find_element(By.XPATH, "//button[@data-testid='download-raw-button']")
    download_source_button.click()

    time.sleep(10)

In [126]:
import os

files = os.listdir(dir)
print(f"{files=}")
name2code = {}

for i, f in enumerate(files):
    if not f.endswith('.tsx'): continue
    code = open(f'{dir}/{f}', "r").read()
    name2code[f] = code

files=['LoginForm.tsx', '.ipynb_checkpoints', 'ProductCard.tsx', 'FullPage.tsx', 'NavigationPanel.tsx']


In [127]:
name2code.keys()

dict_keys(['LoginForm.tsx', 'ProductCard.tsx', 'FullPage.tsx', 'NavigationPanel.tsx'])

In [135]:
from langchain_core.documents import Document

custom_code_docs = [Document(page_content=f'{c.replace(".tsx", "")}\n\nОписание:\n{name2description[c]}\n\nКод:\n{name2code[c]}\n') for c in custom_code_components]
custom_code_ids = [f"{c.replace('.tsx', '')}:{i}" for i, c in enumerate(custom_code_components)]

print(custom_code_docs[1].page_content)

LoginForm

Описание:
Создай форму входа с полями для email и пароля и кнопкой 'Войти'.

Код:
import React, { useState } from "react";
import { Card, Typography, Box, Input, Button } from "@nlmk/ds-2.0";

interface LoginFormProps {
  onSubmit?: (email: string, password: string) => void;
}

const LoginForm: React.FC<LoginFormProps> = ({
  onSubmit = (email, password) =>
    console.log("Login submitted:", { email, password }),
}) => {
  const [email, setEmail] = useState<string>("");
  const [password, setPassword] = useState<string>("");

  const handleSubmit = (e: React.FormEvent) => {
    e.preventDefault();
    onSubmit(email, password);
  };

  return (
    <Card>
      <form onSubmit={handleSubmit}>
        <Box flexDirection="column" gap="var(--24-space)">
          <Typography variant="Heading3">Вход</Typography>
          <Input
            type="email"
            label="Email"
            value={email}
            onChange={(e) => setEmail(e.target.value)}
            required

In [136]:
custom_code_ids

['FullPage:0', 'LoginForm:1', 'NavigationPanel:2', 'ProductCard:3']

In [146]:
docsearch.add_documents(documents=custom_code_docs[1:], ids=custom_code_ids[1:])

Inserting data...: 100%|██████████| 3/3 [00:00<00:00, 9612.61it/s]


['LoginForm:1', 'NavigationPanel:2', 'ProductCard:3']